In [ ]:
def construct_rag_queries(self, state: GraphState) -> GraphState:
    email = state['current_email'].body
    rag_querries = self.agents.generating_querry.invoke({email:email})
    return {'rag_queries':rag_querries.querries}

def retrive_from_rag(self, state: GraphState) -> GraphState:
    final_ans = ""
    for querry in state['rag_queries']:
        ans = self.agents.generate_rag_answer.invoke(querry)
        final_ans += querry + '\n' + ans + '\n\n' 
    return {'retrived_document':final_ans}

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

RAG_SEARCH_PROMPT_TEMPLATE = """
Using the following pieces of retrieved context, answer the question comprehensively and concisely.
Ensure your response fully addresses the question based on the given context.

**IMPORTANT:**
Just provide the answer and never mention or refer to having access to the external context or information in your answer.
If you are unable to determine the answer from the provided context, state 'I don't know.'

Question: {question}
Context: {context}
"""

print("Loading & Chunking Docs...")
loader = TextLoader("agency.txt")
docs = loader.load()

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
doc_chunks = doc_splitter.split_documents(docs)

print("Creating vector embeddings...")
# embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
    )

vectorstore = Chroma.from_documents(doc_chunks, embeddings, persist_directory="db")

# Semantic vector search
vectorstore_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Test RAG chain
print("Test RAG chain...")
prompt = ChatPromptTemplate.from_template(RAG_SEARCH_PROMPT_TEMPLATE)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
llm = ChatNVIDIA(
            model="meta/llama-3.1-70b-instruct",  # choose any NVIDIA-supported model
            )

rag_chain = (
    {"context": vectorstore_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What are your pricing options?"
result = rag_chain.invoke(query)
print(f"Question: {query}")
print(f"Answer: {result}")


c:\Users\DELL\miniconda3\envs\MCP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading & Chunking Docs...
Creating vector embeddings...
Test RAG chain...
Question: What are your pricing options?
Answer: Our pricing options are as follows:

1. **Free Plan**: $0/month, which includes foundational model selection, template prompting, limited versioning, and training with sample data.
2. **Pro Plan**: $49/month, which includes all free plan features, advanced tooling integration, prompt improvement system, enhanced versioning, and training with custom data.
3. **Premium Plan**: $99/month, which includes all pro plan features, priority support, exclusive templates, custom tooling integration, and a dedicated account manager.


In [ ]:
email = 
rag_querries = self.agents.generating_querry.invoke({email:email})
print(rag_querries)